In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from repeng import ControlVector, ControlModel, DatasetEntry

In [3]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = 0

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model = model.to(
    "cuda:0"
    if torch.cuda.is_available()
    else "mps:0"
    if torch.backends.mps.is_available()
    else "cpu"
)
model = ControlModel(model, list(range(-5, -18, -1)))

user_tag, asst_tag = "[INST]", "[/INST]"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
with open("data/all_truncated_outputs.json") as f:
    suffixes = json.load(f)

# you don't need 3 here, you can have as few as one each.
# make sure they are closely matched, however—they should be direct opposites if possible.
# bad: "high on acid" / "sober" — "sober" implies alcohol, so you don't get a clean vector
# good: "high on acid" / "sober, not on acid" — the negative prompt is more directly opposite
positive_personas = ["happy", "ecstatic", "delighted"]
negative_personas = ["sad", "depressed", "dismayed"]


def template(persona: str, suffix: str) -> str:
    return f"{user_tag} Act as if you're extremely {persona}. {asst_tag} {suffix}"


dataset = []
for suffix in suffixes:
    tokens = tokenizer.tokenize(suffix)
    for i in range(1, len(tokens)):
        truncated = tokenizer.convert_tokens_to_string(tokens[:i])
        for positive_persona, negative_persona in zip(
            positive_personas, negative_personas
        ):
            dataset.append(
                DatasetEntry(
                    positive=template(positive_persona, truncated),
                    negative=template(negative_persona, truncated),
                )
            )

# print some example entries
for i in range(3):
    print(f"dataset[{i}].positive:", dataset[i].positive)
    print(f"dataset[{i}].negative:", dataset[i].negative)

dataset[0].positive: [INST] Act as if you're extremely happy. [/INST] That
dataset[0].negative: [INST] Act as if you're extremely sad. [/INST] That
dataset[1].positive: [INST] Act as if you're extremely ecstatic. [/INST] That
dataset[1].negative: [INST] Act as if you're extremely depressed. [/INST] That
dataset[2].positive: [INST] Act as if you're extremely delighted. [/INST] That
dataset[2].negative: [INST] Act as if you're extremely dismayed. [/INST] That


In [5]:
model.reset()  # make sure you always reset the model before training a new vector
control_vector = ControlVector.train(
    model,
    tokenizer,
    dataset,
)

100%|██████████| 31/31 [00:30<00:00,  1.03it/s]


In [7]:
# the question to ask the modified model
# don't forget the space after {user_tag} and before {asst_tag}!
input = f"{user_tag} What are human beings like? {asst_tag}"

# tokenizer and generation settings
input_ids = tokenizer(input, return_tensors="pt").to(model.device)
settings = {
    "pad_token_id": tokenizer.eos_token_id,  # silence warning
    "do_sample": False,  # temperature=0
    "max_new_tokens": 128,
    "repetition_penalty": 1.1,  # reduce control jank
}

print("==baseline")
model.reset()
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n++control")
# add the control vector with a certain strength (try increasing or decreasing this!)
model.set_control(control_vector, 1.5)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n--control")
# subtract the control vector, giving the opposite result (e.g. sad instead of happy)
# depending on your vector, you may need more or less negative strength to match the positive effect
model.set_control(control_vector, -2.0)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))
model.reset()

==baseline
<s> [INST] What are human beings like? [/INST] Human beings are complex and diverse individuals. They have unique personalities, thoughts, feelings, and experiences. They are capable of great love, joy, creativity, and compassion, but also of anger, sadness, fear, and cruelty. They are social creatures who thrive on connection and communication with others, yet they also value their independence and privacy. They are constantly learning and adapting to the world around them, and they strive to find meaning and purpose in their lives. Overall, human beings are a fascinating and intriguing species with endless potential for growth and self-discovery.</s>

++control
<s> [INST] What are human beings like? [/INST] Human beings are incredibly diverse and amazing creatures! They are capable of incredible feats, both good and bad, and can be found all over the world with a wide range of beliefs, cultures, and backgrounds! They are known for their intelligence, creativity, and resour